In [1]:
import numpy as np
np.set_printoptions(precision=3,suppress=True, linewidth=100)


In [2]:
english_alphabet = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','x','w','y','z']
class CTC:
  #y is in the shape (len(alphabet),T)
  def __init__(self,y,label,alphabet = english_alphabet):    
    self.label = label
    self.y = y
    self.init()
  def init(self):
    self.T = self.y.shape[0]
    self.K = self.y.shape[1]
    self.S = len(self.label)*2 + 1
    self.y = np.insert(self.y,0,0,axis=1)
    self.y = np.insert(self.y,0,0,axis=0)
    self.letter_to_index = dict(zip(alphabet,range(1,len(alphabet)+1)))
    self.index_to_letter = dict(zip(range(1,len(alphabet)+1),alphabet))
    self.expanded_label = self.expand_label(self.label)
    self.label.insert(0,"0")
    self.alpha = self.compute_alpha()
    self.beta = self.compute_beta()
    self.grads = self.compute_grads()
    self.loss = self.compute_loss()

  def expand_label(self,label):
    expanded_label = [' ']
    for element in label:
      if(element != ' '):
        expanded_label.append(element)
        expanded_label.append(' ')
    expanded_label.insert(0,"0")
    return expanded_label
    
  def compute_alpha(self):
    
    alpha = np.zeros((self.T + 2, self.S + 2))
    b = self.letter_to_index[' ']
    l1 = self.letter_to_index[self.label[1]]
    alpha[1][1] = self.y[1][b]
    alpha[1][2] = self.y[1][l1]
    
    for t in range(2,self.T + 1):
      for s in range(1, self.S + 1):
        if(s < (self.S - 2*(self.T-t) - 1)):
          continue

        #_s,_t just to make indexing look similar to the original equation..
        _alpha = alpha[t-1][s] + (alpha[t-1][s-1] if s-1 >= 0 else 0 )
         
        ls = self.letter_to_index[self.expanded_label[s]]
        if(self.expanded_label[s] == ' ' or (s >= 2 and self.expanded_label[s-2] == self.expanded_label[s])):
          alpha[t][s] = _alpha * self.y[t][ls]

        else:
          alpha[t][s] = (_alpha + (alpha[t-1][s-2] if s - 2 >= 0 else 0 )) * self.y[t][ls]

        
    

    return alpha

  def compute_beta(self):
  
    beta = np.zeros((self.T + 2,self.S+2))
    b = self.letter_to_index[' ']
    l_last = self.letter_to_index[self.label[-1]]
    beta[self.T][self.S] = self.y[self.T][b]
    beta[self.T][self.S-1] = self.y[self.T][l_last]

    for s in range(self.S, 0,-1):
      for t in range(self.T-1 ,0,-1):
        if(s > 2*t):
          continue
        #_s,_t just to make indexing look similar to the original equation..
        _beta = beta[t+1][s] + beta[t+1][s+1]
        ls = self.letter_to_index[self.expanded_label[s]]

        if(self.expanded_label[s] == ' ' or (s+2 < self.S and self.expanded_label[s+2] == self.expanded_label[s])):
          beta[t][s] = _beta * self.y[t][ls]
        else:
          beta[t][s] = (_beta + beta[t+1][s+2]) * self.y[t][ls]

    return beta


  def compute_loss(self):
    #eq 14
    loss = np.zeros(self.S - 1)
    prod = self.alpha * self.beta
    for s in range(1,self.S+1):
      ls = self.letter_to_index[self.expanded_label[s]]
      loss += prod[1:self.T + 1,s]*self.y[1:self.T + 1,ls]
    
    return np.log(sum(loss))

  def error_signal(self):

    T = self.y.shape[0]
    C = np.sum(self.alpha,axis=1)
    #this should be vectorized for speed.
    alpha_hat = np.array([self.alpha[t]/C[t] for t in range(T)])
    D = np.sum(self.beta,axis=1)
    beta_hat = np.array([self.beta[t]/D[t] for t in range(T)])

    Q = D * np.prod(D/C)
    error = np.zeros(self.y.shape)
    #I think here we better loop over y time steps?
    for element in [' '] + self.label:
      ls = self.letter_to_index[element]
      grads[:,ls] = self.y[:,ls] - Q/self.y[:,ls] * np.sum(alpha_hat*beta_hat,axis=1)
    
    return error

  def compute_grads(self):
    p_lx = self.alpha[self.T,self.S] + self.alpha[self.T,self.S-1]
    grads = np.zeros(self.y.shape)
    
    
    #for all the possible chars in output, if the char is in the label return one, otherwise return 0
    
    #This part needs to be vectorized.
    for t in range(1,self.T + 1):
      for k in range(1,self.K + 1):
        s_sum = 0
        lab = np.nonzero(list(map(lambda x: 1 if self.index_to_letter[k] in x else 0, self.expanded_label)))[0]
        
        for s in lab:
            s_sum += self.alpha[t,s] * self.beta[t,s]
        
        grads[t,k] = (1/p_lx)*(1/(self.y[t,k]**2)) * s_sum
    
    return grads

  def get_alpha(self):
    return self.alpha[1:self.T+1,1:self.S+1]

  def get_beta(self):
    return self.beta[1:self.T+1,1:self.S+1]

  def get_y(self):
    return self.y[1:,1:]

  def get_grads(self):
    return self.grads[1:,1:]

  def get_loss(self):
    return self.loss


In [3]:
y = np.array([[0.173, 0.892, 0.046, 0.966, 0.601, 0.588],
 [0.722, 0.358, 0.922, 0.926, 0.69 , 0.488],
 [0.306 ,0.117, 0.635, 0.872 ,0.167 ,0.053],
 [0.445, 0.387, 0.165, 0.525 ,0.445, 0.081],
 [0.585, 0.398, 0.332 ,0.425, 0.415 ,0.511]]).T
label = ['b','a','b']
alphabet = ['a','b','c','d',' ']

tmp = CTC(y,label,alphabet)

In [4]:
y = tmp.get_y()
print("y: {}\n{}\n".format(y.shape,y))

alpha = tmp.get_alpha()
print("alpha: {}\n{}\n".format(alpha.shape,alpha))

beta = tmp.get_beta()
print("beta: {}\n{}\n".format(beta.shape,beta))

grads = tmp.get_grads()
print("grads: {}\n{}\n".format(grads.shape,grads))

loss = tmp.get_loss()
print("loss: {}\n".format(loss))


y: (6, 5)
[[0.173 0.722 0.306 0.445 0.585]
 [0.892 0.358 0.117 0.387 0.398]
 [0.046 0.922 0.635 0.165 0.332]
 [0.966 0.926 0.872 0.525 0.425]
 [0.601 0.69  0.167 0.445 0.415]
 [0.588 0.488 0.053 0.081 0.511]]

alpha: (6, 7)
[[0.585 0.722 0.    0.    0.    0.    0.   ]
 [0.233 0.468 0.287 0.644 0.    0.    0.   ]
 [0.077 0.646 0.251 0.064 0.214 0.594 0.   ]
 [0.    0.67  0.381 0.929 0.118 0.807 0.252]
 [0.    0.    0.    1.19  0.434 1.279 0.44 ]
 [0.    0.    0.    0.    0.    1.417 0.879]]

beta: (6, 7)
[[0.76  1.535 0.    0.    0.    0.    0.   ]
 [0.601 0.698 0.211 1.217 0.    0.    0.   ]
 [0.09  1.421 0.421 0.108 0.403 0.853 0.   ]
 [0.    0.272 0.125 1.145 0.379 0.835 0.09 ]
 [0.    0.    0.    0.293 0.203 0.689 0.212]
 [0.    0.    0.    0.    0.    0.488 0.511]]

grads: (6, 5)
[[0.    0.926 0.    0.    0.566]
 [0.429 1.111 0.    0.    0.552]
 [1.438 0.73  0.    0.    0.786]
 [0.496 0.435 0.    0.    0.278]
 [0.421 0.807 0.    0.    0.458]
 [0.    1.265 0.    0.    0.749]]

loss: